In [6]:
import cv2
import numpy as np
import os
import glob
import pandas as pd
from PIL import Image 
import mahotas 
import matplotlib.pylab as plt
import csv
import time
start_time = time.time()

#from Features import *

As imagens utilizadas são disponibilizadas no formato DICOM: 

In [8]:
# -*- coding: utf-8 -*-
"""
==============================================================================
@author: Nikolaos Giakoumoglou
@date: Thu May  6 21:59:57 2021
==============================================================================
A.1 First Order Statistics/Statistical Features
==============================================================================
Inputs:
    - f:        image of dimensions N1 x N2
    - mask:     int boolean image N1 x N2 with 1 if pixels belongs to ROI, 
                0 else
Outputs:
    - features: 1)Mean, 2)Variance, 3)Median (50-Percentile), 4)Mode, 
                5)Skewness, 6)Kurtosis, 7)Energy, 8)Entropy, 
                9)Minimal Gray Level, 10)Maximal Gray Level, 
                11)Coefficient of Variation, 12,13,14,15)10,25,75,90-
                Percentile, 16)Histogram width
==============================================================================
"""

import numpy as np

def fos(f, mask):
    
    # 1) Labels
    labels = ["FOS_Mean","FOS_Variance","FOS_Median","FOS_Mode","FOS_Skewness",
              "FOS_Kurtosis","FOS_Energy","FOS_Entropy","FOS_MinimalGrayLevel",
              "FOS_MaximalGrayLevel","FOS_CoefficientOfVariation",
              "FOS_10Percentile","FOS_25Percentile","FOS_75Percentile",
              "FOS_90Percentile","FOS_HistogramWidth"]
    
    # 2) Parameters
    f  = f.astype(np.uint8)
    mask = mask.astype(np.uint8)
    level_min = 0
    level_max = 255
    Ng = (level_max - level_min) + 1
    bins = Ng
    
    # 3) Calculate Histogram H inside ROI
    f_ravel = f.ravel() 
    mask_ravel = mask.ravel() 
    roi = f_ravel[mask_ravel.astype(bool)] 
    H = np.histogram(roi, bins=bins, range=[level_min, level_max], density=True)[0]
    
    # 4) Calculate Features
    features = np.zeros(16,np.double)  
    i = np.arange(0,bins)
    features[0] = np.dot(i,H)
    features[1] = sum(np.multiply(((i-features[0])**2),H))
    features[2] = np.percentile(roi,50) 
    features[3] = np.argmax(H)
    features[4] = sum(np.multiply(((i-features[0])**3),H))/(np.sqrt(features[1])**3)
    features[5] = sum(np.multiply(((i-features[0])**4),H))/(np.sqrt(features[1])**4)
    features[6] = sum(np.multiply(H,H))
    features[7] = -sum(np.multiply(H,np.log(H+1e-16)))
    features[8] = min(roi)
    features[9] = max(roi)
    features[10] = np.sqrt(features[2]) / features[0]
    features[11] = np.percentile(roi,10) 
    features[12] = np.percentile(roi,25)  
    features[13] = np.percentile(roi,75) 
    features[14] = np.percentile(roi,90) 
    features[15] = features[14] - features[11]
    
    return features, labels


In [7]:
##Função que realiza a extrações de características de imagens em uma pasta
def ExtractFeatureDataset(path):
    images_path = os.listdir(path)
    data = []
    for n, image in enumerate(images_path):
        print('Extraindo: ', image, ' Category:', os.path.basename(os.path.normpath(path)), ' Quantidade: ', n, '/', len(images_path))
        img = cv2.imread(os.path.join(path, image))
        #resize_ratio = 0.5
        #img = maintain_aspect_ratio_resize(img, width=int(img.shape[1] * resize_ratio))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.max(2)
        mask = np.ones((img.shape[0], img.shape[1]))
        feature_1 = fos(img, mask)[0]
        print('FOS-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        aux_feature = glcm_features(img)
        feature_2 = np.hstack([aux_feature[0], aux_feature[1]])
        print('GLCM-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        #feature_3 = glds_features(img, mask)[0]
        #print('GLDS-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        #feature_4 = ngtdm_features(img, mask, 8)[0]
        #print('NGTDM-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        #feature_5 = sfm_features(img, mask)[0]
       # print('SGM-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        #feature_6 = lte_measures(img, mask, 3)[0]
        # print('LTE-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        # feature_7 = fps(img, mask)[0]
        # print('FPS-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        # feature_8 = lbp_features_no_mask(img, 8, 2)
        #print('LBP-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        feature_9 = lpq_features(img, 7)
        print('LPQ-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        #feature_10 = cv2.HuMoments(cv2.moments(img_gray)).flatten()
        #print('HUMoments-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        #feature_11 = np.hstack([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7])
        #print('COMBINATION1-Extraido, %s Segundos' % round((time.time() - start_time),2) )
       # feature_12 = np.hstack([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7, feature_8, feature_9, feature_10])
        #print('COMBINATION2-Extraido, %s Segundos' % round((time.time() - start_time),2) )

        imgData = imageData()
        imgData.name = image
        imgData.feature1 = list(feature_1)
        imgData.feature2 = list(feature_2)
        # imgData.feature3 = list(feature_3)
        # imgData.feature4 = list(feature_4)
        # imgData.feature5 = list(feature_5)
        # imgData.feature6 = list(feature_6)
        # imgData.feature7 = list(feature_7)
        # imgData.feature8 = list(feature_8)
        imgData.feature9 = list(feature_9)
        # imgData.feature10 = list(feature_10)
        # imgData.feature11 = list(feature_11)
        # imgData.feature12 = list(feature_12)
        imgData.category = os.path.basename(os.path.normpath(path))
        data.append(imgData)
    return data


In [9]:
# -*- coding: utf-8 -*-
"""
==============================================================================
@author: Nikolaos Giakoumoglou
@date: Thu May  6 19:10:11 2021
@reference: [7] Haralick, Textural Features for Image Classification
==============================================================================
A.2 Gray Level Co-occurence Matrix/Spatial Gray level Difference Matrix
==============================================================================
Inputs:
    - f:             image of dimensions N1 x N2
    - d:             distance to calculate Co-occurence matrix (default d=1)
    - th:            angle to calculate Co-occurence matrix 
                     (default th=[0,45,90,135])
    - ignore_zeros:  ignoore zeros due to mask (default True)
Outputs:
    - features:      Haralick's 1)Angular Second Moment, 2)Contrast, 
                     3)Correlation, 4)Sum of Squares: Variance, 5)Inverse 
                     Difference Moment 6)Sum Average, 7)Sum Variance, 8)Sum 
                     Entropy, 9)Entropy, 10)Difference Variance, 11)Difference 
                     Entropy, 12)Information Measure of Correlation 1, 
                     13)Information Measure of Correlation 2, 14)Maximal 
                     Correlation Coefficient    
==============================================================================
"""
import numpy as np
import mahotas

def glcm_features(f, ignore_zeros=True):
    
    # 1) Labels
    labels = ["GLCM_ASM", "GLCM_Contrast", "GLCM_Correlation",
              "GLCM_SumOfSquaresVariance", "GLCM_InverseDifferenceMoment",
               "GLCM_SumAverage", "GLCM_SumVariance", "GLCM_SumEntropy",
               "GLCM_Entropy", "GLCM_DifferenceVariance",
               "GLCM_DifferenceEntropy", "GLCM_Information1",
               "GLCM_Information2", "GLCM_MaximalCorrelationCoefficient"]
    labels_mean = [label + "_Mean" for label in labels]
    labels_range = [label + "_Range" for label in labels]
    
    # 2) Parameters
    f = f.astype(np.uint8)
    
    # 3) Calculate Features: Mean and Range
    features = mahotas.features.haralick(f, 
                                         ignore_zeros=True, 
                                         compute_14th_feature=True,
                                         return_mean_ptp=True)
    features_mean = features[0:14]
    features_range = features[14:]
    
    return features_mean, features_range, labels_mean, labels_range


In [12]:

from __future__ import division
import cv2

import numpy as np
from scipy.signal import convolve2d

def lpq_features(img,winSize=3,freqestim=1,mode='nh'):
    rho=0.90

    STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
    sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
    sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

    convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

    img=np.float64(img) # Convert np.image to double
    r=(winSize-1)/2 # Get radius from window size
    x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

    if freqestim==1:  #  STFT uniform window
        #  Basic STFT filters
        w0=np.ones_like(x)
        w1=np.exp(-2*np.pi*x*STFTalpha*1j)
        w2=np.conj(w1)

    ## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
    # Run first filter
    filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
    filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
    filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
    filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

    # Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
    freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                        filterResp2.real, filterResp2.imag,
                        filterResp3.real, filterResp3.imag,
                        filterResp4.real, filterResp4.imag])

    ## Perform quantization and compute LPQ codewords
    inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
    LPQdesc=((freqResp>0)*(2**inds)).sum(2)

    ## Switch format to uint8 if LPQ code np.image is required as output
    if mode=='im':
        LPQdesc=np.uint8(LPQdesc)

    ## Histogram if needed
    if mode=='nh' or mode=='h':
        LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

    ## Normalize histogram if needed
    if mode=='nh':
        LPQdesc=LPQdesc/LPQdesc.sum()

    return LPQdesc

In [20]:
class imageData(object):
    __slots__ = ['name', 
                'feature1', 
                'feature2', 
                'feature3', 
                'feature4', 
                'feature5',
                'feature6',
                'feature7',
                'feature8',
                'feature9',
                'feature10',
                'feature11',
                'feature12',
                'category']

img1 = imageData()
img1.name = 'imagem'
folder_path_covid = "C:/Users/junin/Documents/Testecovid"
folder_path_normais = "C:/Users/junin/Documents/Testenormal"
dataCovid = ExtractFeatureDataset(folder_path_covid)
dataNormal = ExtractFeatureDataset(folder_path_normais)
#dataIntersticial = ExtractFeatureDataset(folder_path_intersticiais)
#dataViral = ExtractFeatureDataset(folder_path_viral)

##Combinando para gerar uma lista só para exportarção em CSV
combineData = np.append(dataCovid, dataNormal)
#combineData2 = np.append(combineData, dataIntersticial)
dataSet = combineData

##Titulo de cada atributo
fieldnames = ['Image', 
              'Feature1', 
              'Feature2', 
            #   'Feature3', 
            #   'Feature4', 
            #   'Feature5',
            #   'Feature6',
            #   'Feature7',
            #   'Feature8',
              'Feature9',
            #   'Feature10',
            #   'Feature11',
            #   'Feature12',
              'Category']

##Função que exporta a lista de extrações em CSV 
def WriteCSVFile(path, fieldnames, dataset):
    file = open(path, 'w', newline='', encoding='utf-8')
    writer = csv.writer(file)
    writer.writerow(fieldnames)
    for data in dataset:
        objImg = [data.name, 
                  data.feature1, 
                  data.feature2, 
                #   data.feature3, 
                #   data.feature4, 
                #   data.feature5, 
                #   data.feature6, 
                #   data.feature7, 
                #   data.feature8,
                  data.feature9, 
                #   data.feature10, 
                #   data.feature11, 
                #   data.feature12, 
                  data.category]
        writer.writerow(objImg)

WriteCSVFile('C:/Users/junin/Documents/Kdataset_texture_features_raiox.csv',fieldnames, dataSet)

Extraindo:  Fig -1 -Backes (1).png  Category: Testecovid  Quantidade:  0 / 1
FOS-Extraido, 4647.43 Segundos
GLCM-Extraido, 4647.54 Segundos
LPQ-Extraido, 4647.75 Segundos
Extraindo:  Fig -1 -Backes (1).png  Category: Testenormal  Quantidade:  0 / 1
FOS-Extraido, 4647.81 Segundos
GLCM-Extraido, 4647.92 Segundos
LPQ-Extraido, 4648.12 Segundos
